### Agentic Retriever
- Reduce the cost
- improve the speed

In [ ]:
Enterprise RAG

Souce:  - # of chunks
Product  - 10K
User Guides - 10K
Developer Guides - 10K
QA - 10K
Chat History - 10K

Total - 50K Chunks

Query: 
- How do I create a new feature?
- Which product is best suited for xyz usecase?
- How do I use this feature?



In [1]:
import datasets
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

In [2]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

source_docs = [
    Document(
        page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]}
    ) for doc in knowledge_base
]

docs_processed = RecursiveCharacterTextSplitter(chunk_size=500).split_documents(source_docs)[:1000]

embedding_model = HuggingFaceEmbeddings(model_name="thenlper/gte-small")
vectordb = FAISS.from_documents(
    documents=docs_processed,
    embedding=embedding_model
)

In [35]:
docs_processed[0]

Document(page_content='Create an Endpoint\n\nAfter your first login, you will be directed to the [Endpoint creation page](https://ui.endpoints.huggingface.co/new). As an example, this guide will go through the steps to deploy [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) for text classification. \n\n## 1. Enter the Hugging Face Repository ID and your desired endpoint name:', metadata={'source': 'hf-endpoints-documentation'})

In [36]:
docs_processed[0].metadata

{'source': 'hf-endpoints-documentation'}

In [37]:
n_chunks = len(docs_processed)
print(n_chunks)

1000


In [3]:
all_sources = list(set([doc.metadata["source"] for doc in docs_processed]))
print(all_sources)

['peft', 'gradio', 'diffusers', 'datasets', 'deep-rl-class', 'blog', 'course', 'optimum', 'hub-docs', 'hf-endpoints-documentation', 'evaluate', 'transformers', 'datasets-server', 'pytorch-image-models']


In [34]:
all_sources_2 = list([doc.metadata["source"] for doc in docs_processed])

from collections import Counter
counter = Counter(all_sources_2)
print(counter)

Counter({'transformers': 439, 'deep-rl-class': 74, 'evaluate': 72, 'pytorch-image-models': 69, 'datasets-server': 65, 'gradio': 64, 'blog': 58, 'diffusers': 57, 'course': 38, 'datasets': 32, 'hub-docs': 18, 'hf-endpoints-documentation': 9, 'peft': 3, 'optimum': 2})


In [39]:
counter

Counter({'transformers': 439,
         'deep-rl-class': 74,
         'evaluate': 72,
         'pytorch-image-models': 69,
         'datasets-server': 65,
         'gradio': 64,
         'blog': 58,
         'diffusers': 57,
         'course': 38,
         'datasets': 32,
         'hub-docs': 18,
         'hf-endpoints-documentation': 9,
         'peft': 3,
         'optimum': 2})

In [4]:
import json
from transformers.agents import Tool
from langchain_core.vectorstores import VectorStore

class RetrieverTool(Tool):
    name = "retriever"
    description = "Retrieves some documents from the knowledge base that have the closest embeddings to the input query."
    inputs = {
        "query": {
            "type": "text",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        },
        "source": {
            "type": "text", 
            "description": ""
        },
    }
    output_type = "text"
    
    def __init__(self, vectordb: VectorStore, all_sources: str, **kwargs):
        super().__init__(**kwargs)
        self.vectordb = vectordb
        self.inputs["source"]["description"] = (
            f"The source of the documents to search, as a str representation of a list. Possible values in the list are: {all_sources}. If this argument is not provided, all sources will be searched."
          )

    def forward(self, query: str, source: str = None) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        if source:
            if isinstance(source, str) and "[" not in str(source): # if the source is not representing a list
                source = [source]
            source = json.loads(str(source).replace("'", '"'))

        docs = self.vectordb.similarity_search(query, filter=({"source": source} if source else None), k=3)

        if len(docs) == 0:
            return "No documents found with this filtering. Try removing the source filter."
        return "Retrieved documents:\n\n" + "\n===Document===\n".join(
            [doc.page_content for doc in docs]
        )


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers.agents import HfEngine, ReactJsonAgent

llm_engine = HfEngine("meta-llama/Meta-Llama-3-70B-Instruct")

agent = ReactJsonAgent(
    tools=[RetrieverTool(vectordb, all_sources)],
    llm_engine=llm_engine
)

In [12]:
import os
from openai import OpenAI
from typing import List, Dict
from transformers.agents.llm_engine import MessageRole, get_clean_message_list
from huggingface_hub import InferenceClient


openai_role_conversions = {
    MessageRole.TOOL_RESPONSE: MessageRole.USER,
}


class OpenAIEngine:
    def __init__(self, model_name="gpt-4o-mini"):
        self.model_name = model_name
        self.client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
        )

    def __call__(self, messages, stop_sequences=[]):
        messages = get_clean_message_list(
            messages, role_conversions=openai_role_conversions
        )

        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=messages,
            stop=stop_sequences,
            temperature=0.5,
        )
        return response.choices[0].message.content

In [13]:
#from transformers.agents import HfEngine, ReactJsonAgent

#llm_engine = HfEngine("meta-llama/Meta-Llama-3-70B-Instruct")
#llm_engine = HfEngine("microsoft/Phi-3-mini-4k-instruct")
#llm_engine = HfEngine("mistralai/Mixtral-8x7B-Instruct-v0.1")
llm_engine = OpenAIEngine(model_name="gpt-4o-mini")  


agent = ReactJsonAgent(
    tools=[RetrieverTool(vectordb, all_sources)],
    llm_engine=llm_engine
)

agent_output = agent.run("Please show me a LORA finetuning script")

print("Final output:")
print(agent_output)


======== New task ========
Please show me a LORA finetuning script
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Calling tool: 'retriever' with arguments: {'query': 'LORA finetuning script', 'source': 'blog'}
Calling tool: 'retriever' with arguments: {'query': 'LORA finetuning script', 'source': ''}
Calling tool: 'final_answer' with arguments: You can find a LORA finetuning script at the following link: https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/text_to_image_lora.py


Final output:
You can find a LORA finetuning script at the following link: https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/text_to_image_lora.py


In [24]:
#print(agent.logs) #[2]['llm_output'])

In [14]:
#agent.logs[2] #.keys() #['llm_output']

agent.logs[1]['llm_output']

'Thought: I will search for a document that contains a LORA finetuning script to provide the necessary information.\nAction:\n{\n  "action": "retriever",\n  "action_input": {"query": "LORA finetuning script", "source": "blog"}\n}'

In [15]:
agent.logs[2]['llm_output']

'Thought: Since there were no documents found with the specific source filter, I will search for a LORA finetuning script without any source restrictions.\nAction:\n{\n  "action": "retriever",\n  "action_input": {"query": "LORA finetuning script", "source": ""}\n}'

In [ ]:
['peft', 'gradio', 'diffusers', 'datasets', 'deep-rl-class', 'blog', 'course', 
 'optimum', 'hub-docs', 'hf-endpoints-documentation', 'evaluate', 'transformers', 
 'datasets-server', 'pytorch-image-models']

In [17]:
agent.logs[0]

{'system_prompt': 'You are an expert assistant who can solve any task using JSON tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to the following tools: \'retriever\', \'final_answer\'\nThe way you use the tools is by specifying a json blob, ending with \'<end_action>\'.\nSpecifically, this json should have an `action` key (name of the tool to use) and an `action_input` key (input to the tool).\n\nThe $ACTION_JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. It should be formatted in json. Do not try to escape special characters. Here is the template of a valid $ACTION_JSON_BLOB:\n{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}<end_action>\n\nMake sure to have the $INPUT as a dictionary in the right format for the tool you are using, and do not put variable names as input if you can find the right values.\n\nYou should ALWAYS use the following format:\n\nThought: you should always think 

In [16]:
agent.logs[1]

{'agent_memory': [{'role': <MessageRole.SYSTEM: 'system'>,
   'content': 'You are an expert assistant who can solve any task using JSON tool calls. You will be given a task to solve as best you can.\nTo do so, you have been given access to the following tools: \'retriever\', \'final_answer\'\nThe way you use the tools is by specifying a json blob, ending with \'<end_action>\'.\nSpecifically, this json should have an `action` key (name of the tool to use) and an `action_input` key (input to the tool).\n\nThe $ACTION_JSON_BLOB should only contain a SINGLE action, do NOT return a list of multiple actions. It should be formatted in json. Do not try to escape special characters. Here is the template of a valid $ACTION_JSON_BLOB:\n{\n  "action": $TOOL_NAME,\n  "action_input": $INPUT\n}<end_action>\n\nMake sure to have the $INPUT as a dictionary in the right format for the tool you are using, and do not put variable names as input if you can find the right values.\n\nYou should ALWAYS use the

In [40]:
counter

Counter({'transformers': 439,
         'deep-rl-class': 74,
         'evaluate': 72,
         'pytorch-image-models': 69,
         'datasets-server': 65,
         'gradio': 64,
         'blog': 58,
         'diffusers': 57,
         'course': 38,
         'datasets': 32,
         'hub-docs': 18,
         'hf-endpoints-documentation': 9,
         'peft': 3,
         'optimum': 2})

In [18]:
agent_output = agent.run("How to invoke hf endpoints?")

print("Final output:")
print(agent_output)

======== New task ========
How to invoke hf endpoints?
Calling tool: 'retriever' with arguments: {'query': {'type': 'text', 'description': 'How to invoke Hugging Face endpoints?'}, 'source': 'hf-endpoints-documentation'}
Error in tool call execution: Your search query must be a string
You should only use this tool with a correct input.
As a reminder, this tool's description is the following:

- retriever: Retrieves some documents from the knowledge base that have the closest embeddings to the input query.
    Takes inputs: {'query': {'type': 'text', 'description': 'The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.'}, 'source': {'type': 'text', 'description': "The source of the documents to search, as a str representation of a list. Possible values in the list are: ['peft', 'gradio', 'diffusers', 'datasets', 'deep-rl-class', 'blog', 'course', 'optimum', 'hub-docs', 'hf-endpoints-documentation', 'evaluate', 

Final output:
To invoke Hugging Face endpoints, follow these steps: 1. Create an Endpoint by entering the Hugging Face Repository ID and your desired endpoint name. 2. Deploy a model (e.g., distilbert-base-uncased-finetuned-sst-2-english for text classification). 3. Test your Endpoint using the Inference widget in the overview.


In [19]:
agent.logs[1]['llm_output']

'Thought: I need to find information on how to invoke Hugging Face (hf) endpoints. I will use the `retriever` tool to search for relevant documents that provide guidance on this topic.\nAction:\n{\n  "action": "retriever",\n  "action_input": {"query": {"type": "text", "description": "How to invoke Hugging Face endpoints?"}, "source": "hf-endpoints-documentation"}\n}'

In [22]:
agent.logs[4]['llm_output']

'Thought: I have retrieved some documents that contain information about creating and invoking Hugging Face endpoints. I will now summarize the key steps for invoking these endpoints based on the retrieved information.\nAction:\n{\n  "action": "final_answer",\n  "action_input": "To invoke Hugging Face endpoints, follow these steps: 1. Create an Endpoint by entering the Hugging Face Repository ID and your desired endpoint name. 2. Deploy a model (e.g., distilbert-base-uncased-finetuned-sst-2-english for text classification). 3. Test your Endpoint using the Inference widget in the overview."\n}'